In [1]:
import pandas as pd
import numpy as np

In [2]:
# import movies csv (genre does not matter for v1)
movies_df = pd.read_csv('movielens/movies.csv').drop('genres', axis=1)
print('Shape:', movies_df.shape)
movies_df.head()

Shape: (9125, 2)


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
# import ratings csv (timestamp is not impotant)
ratings_df = pd.read_csv('movielens/ratings.csv').drop('timestamp', axis=1)
print(ratings_df.shape)
ratings_df.head()

(100004, 3)


,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [16]:
num_users = len(ratings_df['userId'].unique())
num_movies = len(movies_df)
print(f"Number of users: {num_users}")
print(f"Number of movies: {num_movies}")

Number of users: 671
Number of movies: 9125


## What's Next?

### Store Movies
- Create a class to store movies
- It should be able to store movies by ID and be able to find its title based on the ID

### Create Users
- From the data we are going to create a bunch of user objects.
- These users will have an ID and they will store the movies they've rated.
- User objects can also be compared, and a similarity score is computed.
  - that metric can be decided on later
    - euclidian distance
    - cosine similarty

### Recommend movies to new users
- Compute the 10 closest neighbors (most similar users) to the new user
- create some list (store) of all movies its nearest neighbors have seen that new user has not seen
- for each of those movies calculate the predicted score the new user would rate it
  - create a score (float) variable = 0
  - create a similarities_sum (float) variable = 0
  - For each neighbor that has seen that movie, add the neighbor's simillarity * their score for the movie
  - divide the score by similarities_sum and return it
- return the top 10 movies you'd recommend the the new user